In [1]:
!pip install pyarrow==16.1.0

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 14.8 MB/s eta 0:00:00
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24.4.1 requires pyarrow<15.0.0a0,>=14.0.1, but you have pyarrow 16.1.0 which is incompatible.
ibis-framework 8.0.0 requires pyarrow<16,>=2, but you have pyarrow 16.1.0 which is incompatible.


In [2]:
!pip install \
    --extra-index-url=https://pypi.nvidia.com \
    cudf-cu12==24.8.* \
    dask-cudf-cu12==24.8.* \
    cuml-cu12==24.8.* \
    cugraph-cu12==24.8.* \
    cuspatial-cu12==24.8.* \
    cuproj-cu12==24.8.* \
    cuxfilter-cu12==24.8.* \
    cucim-cu12==24.8.* \
    pylibraft-cu12==24.8.* \
    raft-dask-cu12==24.8.* \
    cuvs-cu12==24.8.*


Looking in indexes: https://pypi.org/simple, https://pypi.nvidia.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.7/516.7 MB 1.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 GB 994.0 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 GB 954.8 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 152.8/152.8 MB 6.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 915.4/915.4 kB 22.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 83.5/83.5 kB 3.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.7/5.7 MB 84.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 783.8/783.8 MB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.6/190.6 MB 5.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 GB 821.4 kB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [3]:
import cudf
cudf.__version__

'24.08.02'

In [4]:
!python --version

Python 3.10.12


In [5]:
import cuml
cuml.__version__

'24.08.00'

In [6]:
# housing data 불러오기
from pathlib import Path
import pandas as pd
import tarfile
import urllib.request

def load_housing_data():
    tarball_path = Path("datasets/housing.tgz")
    if not tarball_path.is_file():
        Path("datasets").mkdir(parents=True, exist_ok=True)
        url = "https://github.com/ageron/data/raw/main/housing.tgz"
        urllib.request.urlretrieve(url, tarball_path)
        with tarfile.open(tarball_path) as housing_tarball:
            housing_tarball.extractall(path="datasets")
    return pd.read_csv(Path("datasets/housing/housing.csv"))

housing = load_housing_data()
temp_data=pd.DataFrame(housing)
temp_data.fillna(0,inplace=True)

housing data 모델

In [10]:
# 하우싱 데이터 - 그리드 서치

import pandas
import sklearn
import numpy
import cuml
import cudf
import cupy
from cuml import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV # cuml은 랜덤서치가 없다..
from sklearn.metrics import mean_squared_error

X = temp_data.drop('median_house_value', axis=1)
y = temp_data['median_house_value']

# 스케일링은 적당히 조정
def stsc(x) :
    stdsc=StandardScaler()
    temp_x=x.select_dtypes(exclude='object')
    name=temp_x.columns
    stdsc.fit(numpy.array(temp_x).reshape(-1,len(name)))
    temp_x = stdsc.transform(numpy.array(temp_x).reshape(-1, len(name)))
    temp_x=pandas.DataFrame(temp_x)
    temp_x.columns=name
    return temp_x

temp_data.reset_index
temp_x=stsc(X)
temp_x.reset_index

# 모델 설정
cuml_svm = SVR()

# 그리드 서치
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10], # C는 일반적으로 로그스케일 0.1, 1, 10, 100, 1000, ...
    'gamma': [0.01, 0.1, 1] # 일반적으로 1, 0.1, 0.01, 0.001, ...
}

# 그리드 서치 모델 설정
cuml_grid_search = cuml.model_selection.GridSearchCV(cuml_svm, param_grid, cv=5, scoring='neg_root_mean_squared_error', verbose=1)


# Step 1: 80%를 train세트로, 20%를 test 세트로 나눕니다.
x_train, x_test, y_train, y_test = train_test_split(temp_x, y, test_size=0.2, random_state=42)


# 넘파이 배열로 바꿔주기
x_train_np=x_train.to_numpy()
x_test_np=x_test.to_numpy()
y_train_np=y_train.to_numpy()
y_test_np=y_test.to_numpy()


# 모델 적합
cuml_grid_search.fit(x_train_np,y_train_np)

# 평가
print(cuml_grid_search.best_params_)
print(cuml_grid_search.best_score_)

Fitting 5 folds for each of 27 candidates, totalling 135 fits
{'C': 10, 'gamma': 0.1, 'kernel': 'linear'}
-115196.45580038635


In [11]:
# 하우싱 데이터 svm모델 - 랜덤 서치

import pandas
import sklearn
import numpy
import cuml
import cudf
import cupy
from cuml import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV # cuml은 랜덤서치가 없다..
from sklearn.metrics import mean_squared_error

X = temp_data.drop('median_house_value', axis=1)
y = temp_data['median_house_value']

# 스케일링은 적당히 조정
def stsc(x) :
    stdsc=StandardScaler()
    temp_x=x.select_dtypes(exclude='object')
    name=temp_x.columns
    stdsc.fit(numpy.array(temp_x).reshape(-1,len(name)))
    temp_x = stdsc.transform(numpy.array(temp_x).reshape(-1, len(name)))
    temp_x=pandas.DataFrame(temp_x)
    temp_x.columns=name
    return temp_x

temp_data.reset_index
temp_x=stsc(X)
temp_x.reset_index

# 모델 설정
cuml_svm = SVR()

# 랜덤 서치- cuml은 랜덤서치가 없어 사이킷런으로 실행하는데 gpu 사용가능?
param_dist = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': uniform(0.1, 3000),
    'gamma': uniform(0.001, 30)
}

cuml_random_search = RandomizedSearchCV(
    cuml_svm,
    param_distributions=param_dist,
    n_iter=100,  # 무작위로 100개의 조합을 시도
    cv=3,  # 3-폴드 교차 검증
    scoring='neg_root_mean_squared_error',
    verbose=1,
)

# Step 1: 80%를 train세트로, 20%를 test 세트로 나눕니다.
x_train, x_test, y_train, y_test = train_test_split(temp_x, y, test_size=0.2, random_state=42)

# 넘파이 배열로 바꿔주기
x_train_np=x_train.to_numpy()
x_test_np=x_test.to_numpy()
y_train_np=y_train.to_numpy()
y_test_np=y_test.to_numpy()


# 모델 적합
cuml_random_search.fit(x_train_np, y_train_np)

# 평가
print(cuml_random_search.best_params_)
print(cuml_random_search.best_score_)

Fitting 3 folds for each of 100 candidates, totalling 300 fits
{'C': 1337.6733156101557, 'gamma': 0.1790300040797278, 'kernel': 'linear'}
-73127.47274465527


렌딩클럽 모델

In [ ]:
# 렌딩클럽 svm모델 - 그리드 서치
# 위에서 쓴 temp_data 아님

import pandas
import sklearn
import numpy
import cuml
import cudf
import cupy
from cuml import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV # cuml은 랜덤서치가 없다..
from sklearn.metrics import mean_squared_error

X = temp_data.drop('target', axis=1)
y = temp_data['target']

# 스케일링은 적당히 조정
def stsc(x) :
    stdsc=StandardScaler()
    temp_x=x.select_dtypes(exclude='object')
    name=temp_x.columns
    stdsc.fit(numpy.array(temp_x).reshape(-1,len(name)))
    temp_x = stdsc.transform(numpy.array(temp_x).reshape(-1, len(name)))
    temp_x=pandas.DataFrame(temp_x)
    temp_x.columns=name
    return temp_x

temp_data.reset_index
temp_x=stsc(X)
temp_x.reset_index

# 모델 설정
cuml_svm = SVR()

# 그리드 서치
param_grid = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': [0.1, 1, 10], # C는 일반적으로 로그스케일
    'gamma': [0.01, 0.1, 1] # 일반적으로 1, 0.1, 0.01, 0.001, ...
}

# 그리드 서치 모델 설정
cuml_grid_search = cuml.model_selection.GridSearchCV(cuml_svm, param_grid, cv=5, scoring='neg_root_mean_squared_error', verbose=1)

# Step 1: 먼저 80%를 train+validation 세트로, 20%를 test 세트로 나눕니다.
x_train_val, x_test, y_train_val, y_test = train_test_split(temp_x, y, test_size=0.2, random_state=42)

# Step 2: train+validation 세트를 다시 6:2 비율로 train 세트와 validation 세트로 나눕니다.
x_train, x_valid, y_train, y_valid = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=42)

# 넘파이 배열로 바꿔주기
x_train_np=x_train.to_numpy()
x_test_np=x_test.to_numpy()
y_train_np=y_train.to_numpy()
y_test_np=y_test.to_numpy()
x_valid_np=x_valid.to_numpy()
y_valid_np=y_valid.to_numpy()


# 모델 적합
cuml_grid_search.fit(x_train_np,y_train_np)

# 평가
print(cuml_grid_search.best_params_)
print(cuml_grid_search.best_score_)

# Validation 세트에 대한 예측
y_valid_pred = cuml_grid_search.predict(x_valid_np)

# Validation 세트에 대한 평가
validation_rmse = numpy.sqrt(mean_squared_error(y_valid_np, y_valid_pred))
print(f"Validation RMSE: {validation_rmse:.4f}")

In [ ]:
# 렌딩클럽 svm모델 - 랜덤 서치

import pandas
import sklearn
import numpy
import cuml
import cudf
import cupy
from cuml import SVR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from scipy.stats import uniform
from sklearn.model_selection import RandomizedSearchCV # cuml은 랜덤서치가 없다..
from sklearn.metrics import mean_squared_error

X = temp_data.drop('target', axis=1)
y = temp_data['target']

# 스케일링은 적당히 조정
def stsc(x) :
    stdsc=StandardScaler()
    temp_x=x.select_dtypes(exclude='object')
    name=temp_x.columns
    stdsc.fit(numpy.array(temp_x).reshape(-1,len(name)))
    temp_x = stdsc.transform(numpy.array(temp_x).reshape(-1, len(name)))
    temp_x=pandas.DataFrame(temp_x)
    temp_x.columns=name
    return temp_x

temp_data.reset_index
temp_x=stsc(X)
temp_x.reset_index

# 모델 설정
cuml_svm = SVR()

# 랜덤 서치- cuml은 랜덤서치가 없어 사이킷런으로 실행하는데 gpu 사용가능?
param_dist = {
    'kernel': ['linear', 'rbf', 'poly'],
    'C': uniform(0.1, 3000),
    'gamma': uniform(0.001, 30)
}

cuml_random_search = RandomizedSearchCV(
    cuml_svm,
    param_distributions=param_dist,
    n_iter=100,  # 무작위로 100개의 조합을 시도
    cv=3,  # 3-폴드 교차 검증
    scoring='neg_root_mean_squared_error',
    verbose=1,
)

# Step 1: 먼저 80%를 train+validation 세트로, 20%를 test 세트로 나눕니다.
x_train_val, x_test, y_train_val, y_test = train_test_split(temp_x, y, test_size=0.2, random_state=42)

# Step 2: train+validation 세트를 다시 6:2 비율로 train 세트와 validation 세트로 나눕니다.
x_train, x_valid, y_train, y_valid = train_test_split(x_train_val, y_train_val, test_size=0.25, random_state=42)

# 넘파이 배열로 바꿔주기
x_train_np=x_train.to_numpy()
x_test_np=x_test.to_numpy()
y_train_np=y_train.to_numpy()
y_test_np=y_test.to_numpy()
x_valid_np=x_valid.to_numpy()
y_valid_np=y_valid.to_numpy()


# 모델 적합
cuml_random_search.fit(x_train_np, y_train_np)

# 평가
print(cuml_random_search.best_params_)
print(cuml_random_search.best_score_)

# Validation 세트에 대한 예측
y_valid_pred = cuml_random_search.predict(x_valid_np)

# Validation 세트에 대한 평가
validation_rmse = numpy.sqrt(mean_squared_error(y_valid_np, y_valid_pred))
print(f"Validation RMSE: {validation_rmse:.4f}")